In [1]:
import numpy as np
import pandas as pd
import copy

# Load the tickers for the analysis

In [2]:
inp_comp = r'C:\Users\Panagiotis.Mavritsak\Desktop\Reddit sentiment analysis\company data\\'
df_searched = pd.read_csv(inp_comp+'FinalTickers.csv')
# for ticker in df_searched.ticker:
#     print(ticker)
df_searched.ticker.to_list()

['GME',
 'AMC',
 'BB',
 'TSLA',
 'SPY',
 'NOK',
 'CLOV',
 'RKT',
 'WISH',
 'AAPL',
 'AMD',
 'SNDL',
 'SLV',
 'UWMC',
 'TLRY',
 'MVIS',
 'BABA',
 'SPCE',
 'AMZN',
 'CRSR',
 'SOFI',
 'NVDA',
 'CLNE',
 'WKHS',
 'CLF',
 'FB',
 'QQQ',
 'MSFT',
 'RIDE',
 'NFLX',
 'MNMD',
 'SDC',
 'COIN',
 'ICLN',
 'DKNG',
 'RBLX',
 'GOEV',
 'GM',
 'LCID',
 'INTC',
 'DIS',
 'MRNA',
 'AAL',
 'MU',
 'GE',
 'BA',
 'PYPL',
 'JPM',
 'CCL',
 'PFE']

# Organize the RV data for different rolling periods


In [5]:
data_path = r'C:\Users\Panagiotis.Mavritsak\Desktop\Reddit sentiment analysis\Part2\part 2 data\\'
df_RV = pd.read_csv(data_path+'HistoricalVolatility.csv')
df_RV['Timestamp'] = pd.to_datetime(df_RV['date'].astype(str), format='%Y%m%d')
df_RV = df_RV[ df_RV.ticker.isin( df_searched.ticker.to_list() ) ]
df_RV

,secid,date,days,volatility,ticker,index_flag,exchange_d,Timestamp
0,101121,20210104,10,0.182660,AMD,0,5,2021-01-04
1,101121,20210104,14,0.173402,AMD,0,5,2021-01-04
2,101121,20210104,30,0.268824,AMD,0,5,2021-01-04
3,101121,20210104,60,0.365366,AMD,0,5,2021-01-04
4,101121,20210104,91,0.380106,AMD,0,5,2021-01-04
...,...,...,...,...,...,...,...,...
159622,216014,20211231,273,0.695678,SOFI,0,4,2021-12-31
159623,216014,20211231,365,0.695678,SOFI,0,4,2021-12-31
159624,216014,20211231,547,0.695678,SOFI,0,4,2021-12-31
159625,216014,20211231,730,0.695678,SOFI,0,4,2021-12-31


In [7]:
rolling_periods = [10, 14, 30, 60, 91, 122, 152, 182, 273, 365, 547, 730, 1825]
rolling_periods

[10, 14, 30, 60, 91, 122, 152, 182, 273, 365, 547, 730, 1825]

In [8]:
df = pd.DataFrame(data={'Timestamp': np.unique(df_RV.Timestamp)})
df

,Timestamp
0,2021-01-04
1,2021-01-05
2,2021-01-06
3,2021-01-07
4,2021-01-08
...,...
247,2021-12-27
248,2021-12-28
249,2021-12-29
250,2021-12-30


In [6]:
out = r'C:\Users\Panagiotis.Mavritsak\Desktop\Reddit sentiment analysis\Part2\organizedDFs\\'
for period in rolling_periods:
    # unique df for certain period
    df = pd.DataFrame(data={'Timestamp': np.unique(df_RV.Timestamp)})
    # keep df only for specific period
    temp_RV = df_RV[ df_RV.days == period ]
    
    for ticker in df_searched.ticker.to_list():
        ticker_df = temp_RV[temp_RV.ticker==ticker]
        print(f'Analysis for {period}days period for {ticker} with length ', len(ticker_df))
        df[ticker] = np.nan
        # update one by one cause lengths not consistent
        for i in range(len(df)):
#             print(ticker_df[ ticker_df['Timestamp'] == df['Timestamp'].iloc[i] ].volatility.values[0], i)
            if len(ticker_df[ ticker_df['Timestamp'] == df['Timestamp'].iloc[i] ]) > 0:
                df[ticker].iloc[i] = ticker_df[ ticker_df['Timestamp'] == df['Timestamp'].iloc[i] ].volatility.values[0]
            
#         if len(temp_RV[temp_RV.ticker==ticker]) == len(df):
#             df[ticker] = temp_RV[temp_RV.ticker==ticker]
#         else:
    df.to_excel(out+f'RV_{period}days.xlsx', index=False)


# Organize the IV datasets taking the closest strike_price to the actual price

In [10]:
data_path = r'C:\Users\Panagiotis.Mavritsak\Desktop\Reddit sentiment analysis\Part2\part 2 data\\'
df_IV = pd.read_csv(data_path+'ImpliedVolatility.csv')
df_IV['Timestamp'] = pd.to_datetime(df_IV['date'].astype(str), format='%Y%m%d')
df_IV = df_IV[ df_IV.ticker.isin( df_searched.ticker.to_list() ) ]
df_IV

,date,exdate,cp_flag,strike_price,volume,impl_volatility,optionid,ticker,issuer,exercise_style,Timestamp
0,20210104,20210108,C,100000,13744,0.542276,137234995,AMD,ADVANCED MICRO DEVICES INC.,A,2021-01-04
1,20210104,20210108,C,101000,4229,0.553831,137273151,AMD,ADVANCED MICRO DEVICES INC.,A,2021-01-04
2,20210104,20210108,C,102000,3729,0.562234,137273152,AMD,ADVANCED MICRO DEVICES INC.,A,2021-01-04
3,20210104,20210108,C,103000,688,0.569473,137273153,AMD,ADVANCED MICRO DEVICES INC.,A,2021-01-04
4,20210104,20210108,C,104000,376,0.579418,137273154,AMD,ADVANCED MICRO DEVICES INC.,A,2021-01-04
...,...,...,...,...,...,...,...,...,...,...,...
1048570,20210526,20210611,C,2840000,0,0.338802,140071190,AMZN,AMAZON.COM INC.,A,2021-05-26
1048571,20210526,20210611,C,2850000,0,0.344017,140071191,AMZN,AMAZON.COM INC.,A,2021-05-26
1048572,20210526,20210611,C,2860000,0,0.344790,140071192,AMZN,AMAZON.COM INC.,A,2021-05-26
1048573,20210526,20210611,C,2870000,0,0.338216,140071193,AMZN,AMAZON.COM INC.,A,2021-05-26


In [11]:
df_IV['ticker'].unique()

array(['AMD', 'AMZN'], dtype=object)

In [12]:
data_path = r'C:\Users\Panagiotis.Mavritsak\Desktop\Reddit sentiment analysis\Part2\part 2 data\\'
df_price = pd.read_csv(data_path+'StockPrice.csv')
df_price['Timestamp'] = pd.to_datetime(df_price['date'].astype(str), format='%Y%m%d')
df_price = df_price[ df_price.TICKER.isin( df_searched.ticker.to_list() ) ]
df_price.PRC

0         217.69000
1         217.89999
2         212.25000
3         218.28999
4         219.62000
            ...    
15503    1010.64001
15504    1091.83997
15505    1099.56995
15506    1093.98999
15507    1077.59998
Name: PRC, Length: 15249, dtype: float64

In [30]:
out = r'C:\Users\Panagiotis.Mavritsak\Desktop\Reddit sentiment analysis\Part2\organizedDFs\\'
df = pd.DataFrame(data={'Timestamp': np.unique(df_IV.Timestamp)})

# for ticker in df_searched.ticker.to_list():
for ticker in df_IV['ticker'].unique():  # change when we get all tickers
    temp_vol = df_IV[df_IV.ticker==ticker]
    temp_price = df_price[df_price.TICKER==ticker]
    print(f'Analysis for {ticker} with length ', len(temp_vol))
    df[ticker] = np.nan
    # update one by one cause lengths not consistent
    for i in range(len(df)):
        temp_temp_price = temp_price[ temp_price['Timestamp'] == df['Timestamp'].iloc[i] ]
        temp_temp_vol   = temp_vol[   temp_vol['Timestamp']   == df['Timestamp'].iloc[i] ]
        if (len(temp_temp_price) > 0) & (len(temp_temp_vol) > 0):
            df[ticker].iloc[i] = temp_temp_vol['impl_volatility'].loc[temp_temp_vol['strike_price'].sub(temp_temp_price.PRC.values[0]).abs().idxmin()]

df.to_excel(out+f'OrganizedIV.xlsx', index=False)

Analysis for AMD with length  385810


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Analysis for AMZN with length  662765


# Organize the IV datasets taking mean volatility

In [7]:
tickers_for_now = ['GME',
                   'AMC',
                   'BB',
                   'TSLA',
                   'SPY'
                  ]

In [16]:
out = r'C:\Users\Panagiotis.Mavritsak\Desktop\Reddit sentiment analysis\Part2\organizedDFs\\'
df = pd.DataFrame(data={'Timestamp': np.unique(df_RV.Timestamp)})

# different data path for new IV datasets
data_path = r'C:\Users\Panagiotis.Mavritsak\Desktop\Reddit sentiment analysis\Part2\part 2 data\IV per ticker\\'

# for ticker in df_searched.ticker.to_list():
for ticker in tickers_for_now:  # change when we get all tickers
    temp_vol = pd.read_csv(data_path+f'{ticker}.csv')
    temp_vol['Timestamp'] = pd.to_datetime(temp_vol['date'].astype(str), format='%Y%m%d')
    
    print(f'Analysis for {ticker} with length', len(temp_vol))
    df[ticker] = np.nan
    # update one by one cause lengths not consistent
    for i in range(len(df)):
        temp_temp_vol = temp_vol[ temp_vol['Timestamp']==df['Timestamp'].iloc[i] ]
        temp_temp_vol = temp_temp_vol[temp_temp_vol['impl_volatility'].notna()]
        if len(temp_temp_vol) > 0:
            df[ticker].iloc[i] = temp_temp_vol['impl_volatility'].mean()
            
df.to_excel(out+f'MeanIV.xlsx', index=False)

Analysis for GME with length  583116


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Analysis for AMC with length  297781
Analysis for BB with length  179463
Analysis for TSLA with length  1540312
Analysis for SPY with length  2555361
